# Preprocesing 

In [12]:
# https://www.youtube.com/watch?v=kCc8FmEb1nY&ab_channel=AndrejKarpathy
import torch

# Load dataset
with open("shakespear.txt") as fn:
    text = fn.read()

# Get unique lsit
chars = sorted(list(set(text)))

vocab_size = len(chars)
print(chars)
print(len(chars))


['\n', ' ', '!', "'", ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
62


In [2]:
# Tokenize. Chars -> intergers/index
s2i = {ch: i for i, ch in enumerate(chars)}
i2s = {i: ch for i, ch in enumerate(chars)}
encode = lambda x: [s2i[ch] for ch in x]
decode = lambda x: [i2s[i] for i in x]

print(encode("hello"))
print(decode(encode("hello")))


[43, 40, 47, 47, 50]
['h', 'e', 'l', 'l', 'o']


In [ ]:
# Tokenize whole training set
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:1000])

# Train/val
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

torch.Size([99993])
tensor([29, 43, 36, 55,  4,  1, 51, 50, 50, 53,  1, 38, 50, 49, 55, 40, 48, 51,
        55,  4,  1, 50, 53,  1, 38, 47, 36, 44, 48,  3, 39,  1, 55, 43, 50, 56,
         1, 54, 47, 40, 51, 55,  1, 54, 50,  1, 41, 36, 44, 55, 43, 41, 56, 47,
         4,  0, 18,  1, 48, 36, 60,  1, 38, 50, 49, 55, 53, 44, 57, 40,  1, 50,
        56, 53,  1, 41, 36, 55, 43, 40, 53,  8,  1, 36, 49, 39,  4,  1, 44, 49,
         1, 55, 43, 40, 44, 53,  1, 39, 40, 41, 40, 36, 55, 40, 39,  1, 52, 56,
        40, 40, 49,  4,  0, 17, 40, 53,  1, 41, 47, 40, 54, 43,  1, 37, 53, 50,
        46, 40,  1, 48, 40,  1, 36, 49, 39,  1, 51, 56, 55, 55, 36, 49, 38, 40,
         1, 50, 41,  1, 40, 59, 51, 40, 39, 44, 55, 44, 50, 49,  1, 43, 50, 56,
        54, 40,  4,  0, 10, 49, 39,  1, 44, 49,  1, 55, 43, 36, 55,  1, 54, 36,
        48, 40,  1, 55, 43, 36, 55,  1, 40, 57, 40, 53,  1, 18,  1, 47, 36, 48,
        40, 49, 55,  1, 55, 43, 44, 54,  1, 54, 55, 50, 48, 36, 38, 43,  4,  0,
        10, 49, 39, 

How we grab an input:

In [7]:
context_length = 8
print(train_data[: context_length + 1])

tensor([29, 43, 36, 55,  4,  1, 51, 50, 50])


Example of how it's auto regressive

In [6]:
x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length):
    context = x[:t+1]
    target = y[t]
    print(f"using input {context} the target is : {target}")
    

using input tensor([29]) the target is : 43
using input tensor([29, 43]) the target is : 36
using input tensor([29, 43, 36]) the target is : 55
using input tensor([29, 43, 36, 55]) the target is : 4
using input tensor([29, 43, 36, 55,  4]) the target is : 1
using input tensor([29, 43, 36, 55,  4,  1]) the target is : 51
using input tensor([29, 43, 36, 55,  4,  1, 51]) the target is : 50
using input tensor([29, 43, 36, 55,  4,  1, 51, 50]) the target is : 50


sample minibatch: notice input encodes autoregressive style inputs

In [10]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
context_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_size, (batch_size,))
    x = torch.stack([data[i:i+context_size] for i in ix])
    y = torch.stack([data[i+1:i+context_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(context_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[36, 45, 40, 54, 55, 60,  6,  0],
        [47, 47,  1, 42, 50,  4,  0, 10],
        [18,  1, 54, 40, 40,  0, 15, 50],
        [49, 50,  1, 51, 53, 50, 51, 43]])
targets:
torch.Size([4, 8])
tensor([[45, 40, 54, 55, 60,  6,  0,  0],
        [47,  1, 42, 50,  4,  0, 10, 49],
        [ 1, 54, 40, 40,  0, 15, 50, 53],
        [50,  1, 51, 53, 50, 51, 43, 40]])
----
when input is [36] the target: 45
when input is [36, 45] the target: 40
when input is [36, 45, 40] the target: 54
when input is [36, 45, 40, 54] the target: 55
when input is [36, 45, 40, 54, 55] the target: 60
when input is [36, 45, 40, 54, 55, 60] the target: 6
when input is [36, 45, 40, 54, 55, 60, 6] the target: 0
when input is [36, 45, 40, 54, 55, 60, 6, 0] the target: 0
when input is [47] the target: 47
when input is [47, 47] the target: 1
when input is [47, 47, 1] the target: 42
when input is [47, 47, 1, 42] the target: 50
when input is [47, 47, 1, 42, 50] the target: 4
when input is [47, 

# Bigram model

In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers. get the embeddings. 
        # each index in input tensor takes out token_embeeding_table row
        logits = self.token_embedding_table(idx) # (B,T,C)
        B,T,C = logits.shape
        logits = logits.view(B*T, C)
        targets=targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss) # expecting ~ -ln(1/65)

torch.Size([32, 62])
tensor(4.8775, grad_fn=<NllLossBackward0>)


Add a generative step. 

In [18]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
            
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None: # targets optional for inference 
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            
            # get most recent token 
            logits = logits[:, -1, :] # becomes (B, C)
            
            # softmax over vocabulary 
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution to get next input 
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 62])
tensor(4.4976, grad_fn=<NllLossBackward0>)
['\n', 'R', 'H', 'F', 'Z', 'T', 'B', 'p', 'a', 'L', 'M', 'D', 'q', '?', 'r', 'p', ':', 'x', 'P', ';', '\n', 'd', 'M', 'e', 'g', "'", 'G', 'Z', 'Y', 'J', 'y', 'D', ' ', 'R', 'f', 'S', 'l', 'X', 'k', 'H', 'q', 'r', 'E', 'J', 'X', ';', 'Z', '-', 'M', 'Z', 'b', 'C', 'v', 'W', 'j', 'm', 'B', 'V', 'o', 'z', 'h', 'Q', 'O', 's', ':', '-', 'i', '\n', 'b', '.', 'r', 'u', 'c', 'O', 'x', 'g', 'W', 'd', 'h', 'r', 'E', 'Z', 'g', 'w', '\n', 'r', 'a', 'Y', ':', 'i', 'Y', ',', ',', 'n', 'K', 'N', ',', ':', '.', 'U', 'C']


In [40]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # 3e-4 for big network. 1e-3 for small networks.  

In [41]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
                    

2.3742942810058594


In [42]:
print("".join(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist())))


Frootorst thoshe thedy
M?

Thato heto llu stweepano abod tl.thet gmpouVEvinge th.
Anavemyondurgse I wat
An, y geacofr teite; s e cuthay artheng,-kes eqXHAT:


NShin! we wous send.
And tound limed.
Istoe ifrtyo honsete Ypofroavisour bunk ome 'dedYond CRKEDuee be te.
apisbusoirulo oqbld fref ayed itha s,
I shireeans tends od ingure sidss llcond,
l be to iarrkiththere
ABur s, se htis k.
Sto an obun.
poraturcitothanssh,
MELod
sthexanje sthe
Timay pa fex: roDirexck y io ie s fo, momyousid w,

R: be t


In [ ]:
@torch.no_grad() # no storing grad chain. no backward calls 
def estimate_loss():
    pass




### Trick for self attention



In [44]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

x[b,t] = mean_{z<=t} x[b,i]... take mean feature vector of history

In [48]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [49]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [50]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

first is first. second is average of first and second. third is average of frist, second and third

can achieve this loop efficiently with a lower triangular matrix that is normalised along the rows

In [52]:
a = torch.tril(torch.ones(3,3)) 
a = a/ torch.sum(a, 1, keepdim=True)
a

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [54]:
# for above 
a = torch.tril(torch.ones(T,T)) 
a = a/ torch.sum(a, 1, keepdim=True)
xbow2 = a @ x # (B,T,T) @ (B,T,C) ---> (B,T,C)
xbow2

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [55]:
torch.allclose(xbow, xbow2)

True

The idea is to switch to a weighted sum where weighting is attention. So this weghting will be learnt. 